In [42]:
import pyomo.environ as pe  # Pyomo environment
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95
import numpy as np


In [43]:
# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

temperatures taken from Cinti
,2020

In [44]:
# Provided data in Celsius, convert to Kelvin
T_hot_inlet_C = 662.30
T_hot_outlet_C = 192.60
T_cold_inlet_C = 20.00
T_cold_outlet_C = 496.94


T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin


Check these!

In [45]:

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.039  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air at constant pressure in kJ/kg·K


Understand where we use this ref temperature

In [46]:

# Reference temperature (for simplicity, assume 0°C = 273.15 K)
T_ref = 273.15

In [47]:
# Calculate enthalpy using Cp (simplified)
def calc_enthalpy(Cp, T):
    return Cp * (T - T_ref)

In [48]:
# Calculate enthalpy for the hot and cold streams
h_hot_inlet = calc_enthalpy(Cp_hot, T_hot_inlet)  # Inlet enthalpy for the hot stream (N2/steam)
h_hot_outlet = calc_enthalpy(Cp_hot, T_hot_outlet)
h_cold_inlet = calc_enthalpy(Cp_cold, T_cold_inlet)  # Inlet enthalpy for the cold stream (air)
h_cold_outlet = calc_enthalpy(Cp_cold, T_cold_outlet)

In [49]:
# Mass flow rate (Assuming constant flow rate of 100 mol/s)
flow_mol_hot = 100  # mol/s
flow_mol_cold = 100  # mol/s

In [50]:
# Convert molar flow to mass flow (assuming ideal gas behavior)
M_hot = 28.0134  # Molar mass of N2 (kg/kmol)
M_cold = 28.97  # Molar mass of air (kg/kmol)

In [51]:
mass_flow_hot = flow_mol_hot * M_hot / 1000  # Convert to kg/s
mass_flow_cold = flow_mol_cold * M_cold / 1000  # Convert to kg/s

In [52]:
# Compute heat duty (Q)
Q_hot = mass_flow_hot * Cp_hot * (T_hot_inlet - T_hot_outlet)  # kJ/s = kW
Q_cold = mass_flow_cold * Cp_cold * (T_cold_outlet - T_cold_inlet)  # kJ/s = kW

In [53]:
# Ensure heat balance is satisfied
Q = min(Q_hot, Q_cold) * 1000  # Convert kW to W

In [54]:
# Compute LMTD
delta_T1 = T_hot_inlet - T_cold_outlet
delta_T2 = T_hot_outlet - T_cold_inlet
LMTD = (delta_T1 - delta_T2) / np.log(delta_T1 / delta_T2)

In [55]:
# Fixed overall heat transfer coefficient
U = 100  # W/m²·K

In [56]:

# Compute heat exchanger area
A = Q / (U * LMTD)

In [57]:
# **Store these values as variables for visualization**
model.fs.heat_duty = pe.Param(initialize=Q, doc="Heat Duty (W)")  # Heat Duty
model.fs.LMTD = pe.Param(initialize=LMTD, doc="Log Mean Temperature Difference (K)")  # LMTD
model.fs.area_calculated = pe.Param(initialize=A, doc="Calculated Heat Exchanger Area (m²)")

In [70]:
model.fs.heat_exchanger.area = A  # Set the calculated area to the heat exchanger model

In [71]:
# Create the Heat Exchanger model
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

(type=<class 'idaes.core.base.process_block._ScalarHeatExchanger'>) on block
fs with a new Component (type=<class
'idaes.core.base.process_block._ScalarHeatExchanger'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [72]:
# Fixing inlet flow and pressure values (assuming standard conditions)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(100)  # Hot stream flow rate (mol/s)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)  # Standard atmospheric pressure
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(100)  # Cold stream flow rate (mol/s)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)  # Standard atmospheric pressure

In [77]:
# Fixing enthalpies for the heat exchanger
model.fs.heat_exchanger.shell_inlet.enth_mol[0].set_value(h_hot_inlet)  # Hot stream inlet enthalpy
model.fs.heat_exchanger.shell_outlet.enth_mol[0].set_value(h_hot_outlet)  # Hot stream outlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol[0].set_value(h_cold_inlet)  # Cold stream inlet enthalpy
model.fs.heat_exchanger.tube_outlet.enth_mol[0].set_value(h_cold_outlet)  # Cold stream outlet enthalpy

In [79]:
# **Store these values as variables for visualization**
model.fs.heat_duty = pe.Param(initialize=Q, doc="Heat Duty (W)")  # Heat Duty
model.fs.LMTD = pe.Param(initialize=LMTD, doc="Log Mean Temperature Difference (K)")  # LMTD
model.fs.area_calculated = pe.Param(initialize=A, mutable=True, doc="Calculated Heat Exchanger Area (m²)")  # Area


'pyomo.core.base.param.ScalarParam'>) on block fs with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.param.ScalarParam'>) on block fs with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.ScalarParam'>) on block fs with a new
Component (type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [84]:
# **Assign the calculated area to the Heat Exchanger's area**
model.fs.heat_exchanger.area = A  # Set the calculated area to the heat exchanger model

In [86]:

# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

InitializationError: Degrees of freedom for fs.heat_exchanger were not equal to zero during initialization (DoF = 2).

In [83]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 


# Visualize the model
visualize(model.fs, "HeatExchangerAreaCalculation")

2025-03-28 15:52:56 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 63203
2025-03-28 15:52:56 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerAreaCalculation.json'
2025-03-28 15:52:56 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaCalculation.json' in current directory (c:\Users\Sara\Desktop\IDAES\HE)
2025-03-28 15:52:56 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:63203/app?id=HeatExchangerAreaCalculation


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x000001731896A6B0>, port=63203, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x000001732216C460>)

In [87]:
# Print results
print(f"Heat Duty (Q): {Q / 1000:.2f} kW")
print(f"LMTD: {LMTD:.2f} K")
print(f"Calculated Heat Exchanger Area: {A:.2f} m²")

Heat Duty (Q): 1367.11 kW
LMTD: 168.95 K
Calculated Heat Exchanger Area: 80.92 m²
